In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.lm_rnn import *
from torchtext import data
from fastai.nlp import *

C:\Users\Matt Wilde\AppData\Local\conda\conda\envs\fastai\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Matt Wilde\AppData\Local\conda\conda\envs\fastai\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\Matt Wilde\AppData\Local\conda\conda\envs\fastai\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [4]:
?data.TabularDataset

In [5]:
?data.Field

In [19]:
import pandas as pd
df=pd.read_csv('data/toxiccomments/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [28]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [33]:
with open('data/toxiccomments/trn/trn.txt', 'w', encoding='utf-8') as file:
    file.write(train['comment_text'].str.cat(sep=' '))
    
with open('data/toxiccomments/val/val.txt', 'w', encoding='utf-8') as file:
    file.write(test['comment_text'].str.cat(sep=' '))

In [2]:
TEXT = data.Field(lower=True, tokenize="spacy")

batch_size=64
bptt=64
n_embedding=128
n_hidden=256

PATH='data/toxiccomments/'
FILES = dict(train='trn/', validation='val/', test='val/')
model_data = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=batch_size, bptt=bptt, min_freq=3)
len(model_data.trn_dl)

C:\Users\Matt Wilde\AppData\Local\conda\conda\envs\fastai\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\Matt Wilde\AppData\Local\conda\conda\envs\fastai\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


2565

In [3]:
model_data.nt

57046

In [4]:
class SimpleLM(nn.Module):
    def __init__(self, vocab_size, n_embedding, batch_size, n_layers, n_hidden):
        super().__init__()
        self.vocab_size, self.n_layers, self.n_hidden = vocab_size, n_layers, n_hidden
        self.embedding = nn.Embedding(vocab_size, n_embedding)
        self.rnn = nn.LSTM(n_embedding, n_hidden, n_layers, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(batch_size)
    
    def init_hidden(self, batch_size):
        self.h = (
            V(torch.zeros(self.n_layers, batch_size, n_hidden)), 
            V(torch.zeros(self.n_layers, batch_size, n_hidden)),
        )
    
    def forward(self, inputs):
        batch_size = inputs[0].size(0)
        if self.h[0].size(1) != batch_size:
            self.init_hidden(batch_size)
        output, h = self.rnn(self.embedding(inputs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(output), dim=-1).view(-1, self.vocab_size)

In [5]:
model = SimpleLM(model_data.nt, n_embedding, batch_size, 2, n_hidden).cuda()

In [6]:
optimizer = LayerOptimizer(optim.Adam, model, 1e-3, 1e-6)

In [7]:
fit(model, model_data, 2, optimizer.opt, F.nll_loss)

epoch      trn_loss   val_loss                                                                                         
    0      5.333268   5.26539   
    1      5.001659   4.953765                                                                                         



[array([4.95377])]

In [9]:
save_model(model, f'{PATH}models/lm_2')

In [10]:
fit(model, model_data, 4, optimizer.opt, F.nll_loss)

epoch      trn_loss   val_loss                                                                                         
    0      4.867361   4.810172  
    1      4.773617   4.72621                                                                                          
    2      4.673922   4.671996                                                                                         
    3      4.662387   4.627062                                                                                         



[array([4.62706])]

In [11]:
save_model(model, f'{PATH}models/lm_3')

In [12]:
fit(model, model_data, 4, optimizer.opt, F.nll_loss)

epoch      trn_loss   val_loss                                                                                         
    0      4.582664   4.592357  
    1      4.538786   4.562852                                                                                         
    2      4.567875   4.539281                                                                                         
    3      4.486047   4.520083                                                                                         



[array([4.52008])]

In [13]:
save_model(model, f'{PATH}models/lm_4')

In [14]:
fit(model, model_data, 8, optimizer.opt, F.nll_loss)

epoch      trn_loss   val_loss                                                                                         
    0      4.503924   4.495605  
    1      4.445376   4.483379                                                                                         
    2      4.431187   4.470992                                                                                         
    3      4.412627   4.459268                                                                                         
    4      4.395597   4.445696                                                                                         
    5      4.378091   4.437461                                                                                         
    6      4.375101   4.427366                                                                                         
    7      4.365937   4.4211                                                                                           



[array([4.4211])]

In [15]:
save_model(model, f'{PATH}models/lm_5')

In [23]:
class LMClassifier(nn.Module):
    def __init__(self, language_model, batch_size, n_classes):
        super().__init__()
        self.language_model = language_model
        self.n_classes = n_classes
        self.n_layers = language_model.n_layers
        self.n_hidden = language_model.n_hidden
        self.embedding = language_model.embedding
        self.rnn = language_model.rnn
        self.l_out = nn.Linear(n_hidden, n_classes)
        self.init_hidden(batch_size)
    
    def init_hidden(self, batch_size):
        self.h = (
            V(torch.zeros(self.n_layers, batch_size, n_hidden)), 
            V(torch.zeros(self.n_layers, batch_size, n_hidden)),
        )
    
    def forward(self, inputs):
        batch_size = inputs[0].size(0)
        if self.h[0].size(1) != batch_size:
            self.init_hidden(batch_size)
        output, h = self.rnn(self.embedding(inputs), self.h)
        self.h = repackage_var(h)
        return F.sigmoid(self.l_out(output), dim=-1).view(-1, self.n_classes)

In [24]:
classifier_model = LMClassifier(model, batch_size, 6).cuda()

In [ ]:
ModelData